# Main Goals of the project
- predict 'taxvaluedollarcount'
    - on single unit properties
    - between May and June of 2017
- what states and counties are properties located
- plot distribution of tax rates for each county

For the first iteration of your model, use only square feet of the home, number of bedrooms, and number of bathrooms to estimate the property's assessed value, taxvaluedollarcnt. You can expand this to other fields after you have completed an mvp (minimally viable product).

#####  single family homes in may and june
select propertylandusetypeid, count(propertylandusetypeid)

from properties_2017 as prop 

join predictions_2017 as pred on pred.parcelid = prop.parcelid

where transactiondate BETWEEN '2017-05-01' AND '2017-06-30'

and propertylandusetypeid not in (247, 246, 269, 248, 31)

group by propertylandusetypeid;

##### bare minimum values between may and june
select calculatedfinishedsquarefeet, bedroomcnt, bathroomcnt, transactiondate, propertylandusetypeid

from properties_2017 as prop 

join predictions_2017 as pred on pred.parcelid = prop.parcelid

where transactiondate BETWEEN '2017-05-01' AND '2017-06-30';

##### checking for nulls and values 
select calculatedfinishedsquarefeet, bedroomcnt, bathroomcnt
from properties_2017

where bathroomcnt is not null 

and bathroomcnt > 0

and bedroomcnt is not null 

and bedroomcnt > 0

and calculatedfinishedsquarefeet is not null

and calculatedfinishedsquarefeet > 0;